In [1]:
import numpy as np

In [122]:
with open('input.txt', 'r') as f:
    data = f.read().splitlines()

obstacles = []
visited = []
guard = (0, 0)
height = len(data)
width = len(data[0])

for y, line in enumerate(data):
    for x, char in enumerate(line):
        if char == '#':
            obstacles.append((x, y))
        if char == '^':
            guard = (x, y)
            visited.append((x, y))
            direction = (0, -1)


print(obstacles)
print(guard)
print(direction)

[(0, 0), (6, 0), (16, 0), (42, 0), (46, 0), (67, 0), (74, 0), (82, 0), (94, 0), (103, 0), (18, 1), (25, 1), (60, 1), (79, 1), (84, 1), (100, 1), (102, 1), (123, 1), (129, 1), (14, 2), (26, 2), (33, 2), (35, 2), (75, 2), (94, 2), (101, 2), (0, 3), (1, 3), (33, 3), (44, 3), (64, 3), (72, 3), (121, 3), (0, 4), (22, 4), (30, 4), (31, 4), (48, 4), (52, 4), (63, 4), (108, 4), (112, 4), (117, 4), (8, 5), (17, 5), (31, 5), (47, 5), (51, 5), (52, 5), (72, 5), (84, 5), (96, 5), (104, 5), (116, 5), (117, 5), (119, 5), (123, 5), (125, 5), (48, 6), (101, 6), (103, 6), (9, 7), (14, 7), (84, 7), (85, 7), (92, 7), (98, 7), (107, 7), (123, 7), (29, 8), (35, 8), (45, 8), (68, 8), (72, 8), (89, 8), (91, 8), (102, 8), (40, 9), (114, 9), (41, 10), (49, 10), (63, 10), (65, 10), (91, 10), (96, 10), (102, 10), (114, 10), (3, 11), (16, 11), (17, 11), (53, 11), (65, 11), (106, 11), (125, 11), (4, 12), (22, 12), (30, 12), (43, 12), (49, 12), (61, 12), (94, 12), (96, 12), (98, 12), (126, 12), (3, 13), (68, 13), (

In [118]:
def get_dicts(coords, height: int, width: int) -> tuple[dict, dict]:
    col_dict = {}
    for i in coords:
        if i[0] not in col_dict:
            col_dict[i[0]] = []
        col_dict[i[0]].append(i[1])

    for i in col_dict:
        col_dict[i] = sorted(col_dict[i])

    row_dict = {}
    for i in coords:
        if i[1] not in row_dict:
            row_dict[i[1]] = []
        row_dict[i[1]].append(i[0])
    
    for i in row_dict:
        row_dict[i] = sorted(row_dict[i])

    for i in range(height):
        if i not in row_dict:
            row_dict[i] = []
    for i in range(width):
        if i not in col_dict:
            col_dict[i] = []
    
    return col_dict, row_dict


In [119]:
def binary_search(arr, target):
    left, right = 0, len(arr) - 1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    return -1

def search_insert_position(arr, target):
    left, right = 0, len(arr)
    while left < right:
        mid = (left + right) // 2
        if arr[mid] < target:
            left = mid + 1
        else:
            right = mid
    return left

In [102]:
def is_obstacle(coord):
    for obstacle in obstacles:
        if coord == obstacle:
            return True
    return False

def turn_right(direction) -> tuple:
    if direction == (0, -1):
        return (1, 0)
    if direction == (1, 0):
        return (0, 1)
    if direction == (0, 1):
        return (-1, 0)
    if direction == (-1, 0):
        return (0, -1)
    
def is_vertical(direction) -> bool:
    if direction == (0, -1) or direction == (0, 1):
        return True
    return False
    
def is_out_of_bounds(coord: tuple[int, int]) -> bool:
    if coord[0] < 0 or coord[0] >= height:
        return True
    if coord[1] < 0 or coord[1] >= width:
        return True
    return False

def get_all_coords_between_including_edge(coord1: tuple, coord2: tuple):
    if coord1[0] == coord2[0]:
        if coord1[1] < coord2[1]:
            return [(coord1[0], i) for i in range(coord1[1], coord2[1] + 1)]
        else:
            return [(coord1[0], i) for i in range(coord2[1], coord1[1] + 1)]
    else:
        if coord1[0] < coord2[0]:
            return [(i, coord1[1]) for i in range(coord1[0], coord2[0] + 1)]
        else:
            return [(i, coord1[1]) for i in range(coord2[0], coord1[0] + 1)]
    

def add_to_visited(coord: tuple, new_coord: tuple, v: list[tuple]):
    # add all the coordinates between coord and new_coord to visited    
    for i in get_all_coords_between_including_edge(coord, new_coord):
        if i not in v:
            v.append(i)



def walk(coord, direction, row_dict: dict[int, list[int]], col_dict: dict[int, list[int]]) -> tuple[bool, tuple, tuple]:
    # new_coord = (coord[0] + direction[0], coord[1] + direction[1])
    if is_vertical(direction):
        idx = search_insert_position(col_dict[coord[0]], coord[1])
        if direction == (0, -1):  # UP
            if col_dict[coord[0]] is not None and idx > 0 and idx <= len(col_dict[coord[0]]):
                next_obstacle_y = col_dict[coord[0]][idx-1]
                return True, (coord[0], next_obstacle_y + 1), turn_right(direction)
            return False, (coord[0], -1), direction
        elif direction == (0, 1):  # DOWN
            if col_dict[coord[0]] is not None and idx < len(col_dict[coord[0]]):
                next_obstacle_y = col_dict[coord[0]][idx]
                return True, (coord[0], next_obstacle_y - 1), turn_right(direction)
            return False, (coord[0], height), direction
        else:
            raise ValueError('Invalid direction')
    else:
        idx = search_insert_position(row_dict[coord[1]], coord[0])
        if direction == (1, 0):  # RIGHT
            if row_dict[coord[1]] is not None and idx < len(row_dict[coord[1]]):
                next_obstacle_x = row_dict[coord[1]][idx]
                return True, (next_obstacle_x - 1, coord[1]), turn_right(direction)
            return False, (width, coord[1]), direction
        elif direction == (-1, 0):  # LEFT
            if row_dict[coord[1]] is not None and idx > 0 and idx <= len(row_dict[coord[1]]):
                next_obstacle_x = row_dict[coord[1]][idx-1]
                return True, (next_obstacle_x + 1, coord[1]), turn_right(direction)
            return False, (-1, coord[1]), direction
        else:
            print(direction)
            raise ValueError('Invalid direction')
        
# col_dict, row_dict = get_dicts(obstacles, height, width)
# print(col_dict)

def start_walk(coord, dir) -> tuple[tuple, tuple]:
    col_dict, row_dict = get_dicts(obstacles, height, width)
    in_range = True
    visited = []
    while in_range:
        in_range, new_coord, new_dir = walk(coord, dir, col_dict=col_dict, row_dict=row_dict)
        add_to_visited(coord, new_coord, visited)
        coord = new_coord
        dir = new_dir
        print(in_range, coord, dir)
    return visited
        
vis = start_walk(guard, direction)
print(len(vis))

True (4, 1) (1, 0)
True (8, 1) (0, 1)
True (8, 6) (-1, 0)
True (2, 6) (0, -1)
True (2, 4) (1, 0)
True (6, 4) (0, 1)
True (6, 8) (-1, 0)
True (1, 8) (0, -1)
True (1, 7) (1, 0)
True (7, 7) (0, 1)
False (7, 10) (0, 1)
42


In [127]:
import copy

def add_to_visited(coord: tuple, new_coord: tuple, v: list[tuple[int, int]]):
    # add all the coordinates between coord and new_coord to visited    
    for i in get_all_coords_between_including_edge(coord, new_coord):
        if i not in v:
            v.append(i)

def detect_loop_forward(coord, dir, collision_col_dict, collision_row_dict, v: list[tuple]) -> list[tuple[int, int]]:
    in_range, new_coord, new_dir = walk(coord, dir, col_dict=collision_col_dict, row_dict=collision_row_dict)
    coords: list[tuple] = get_all_coords_between_including_edge(coord, new_coord)
    possible_obstacles = []
    for o in coords:
        if o in v or o == coord or o[0] >= height or o[1] >= width or o[0] < 0 or o[1] < 0:
            continue
        new_row_dict = copy.deepcopy(collision_row_dict)
        new_col_dict = copy.deepcopy(collision_col_dict)
        new_row_dict[o[1]].append(o[0])
        new_col_dict[o[0]].append(o[1])
        new_row_dict[o[1]] = sorted(new_row_dict[o[1]])
        new_col_dict[o[0]] = sorted(new_col_dict[o[0]])


        dir_copy = dir
        coord_copy = coord
        in_range_new = True

        loop_visited = []
        dir_visited = []
        while in_range_new:
            if coord_copy not in loop_visited:
                loop_visited.append(coord_copy)
                dir_visited.append(dir_copy)

            in_range_new, coord_copy, dir_copy = walk(coord_copy, dir_copy, col_dict=new_col_dict, row_dict=new_row_dict)
            if not in_range_new:
                break
            if coord_copy in loop_visited:
                idx = loop_visited.index(coord_copy)
                if dir_visited[idx] == dir_copy:
                    if o not in possible_obstacles:
                        possible_obstacles.append(o)
            # if coord_copy == coord and dir == dir_copy:
            #     if o not in possible_obstacles:
            #         possible_obstacles.append(o)
            #     break
                    break

    return possible_obstacles
    
def start_walk_v2(coord, dir) -> tuple[tuple, tuple]:
    col_dict, row_dict = get_dicts(obstacles, height, width)
    in_range = True
    possible_obstacles = []
    visited = []
    while in_range:
        forward_blocks = detect_loop_forward(coord, dir, col_dict, row_dict, visited)
        possible_obstacles.extend(forward_blocks)
        in_range, new_coord, new_dir = walk(coord, dir, col_dict=col_dict, row_dict=row_dict)
        add_to_visited(coord, new_coord, visited)
        coord = new_coord
        dir = new_dir
    print(len(possible_obstacles))
        

start_walk_v2(guard, direction)

1670


In [138]:
def find(arr, target):
    left, right = 0, len(arr) - 1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == target:
            return True
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    return False


In [139]:
for i in range(height):
    if i not in row_dict:
        row_dict[i] = []

for i in range(width):
    if i not in col_dict:
        col_dict[i] = []

In [140]:
(4,6) in visited, (4,6) == (4,6)

(True, True)

In [141]:
for y in range(height):
    for x in range(width):
        if find(col_dict[x], y):
            # loop through all x values 1 row lower and to the right
            for i in range(x+1, width):
                if find(row_dict[y], i):
                    print('X', end='')
                else:
                    print('.', end='')
        else:
            print('.', end='')
    print()

..............
.........
..........
................
...........
..........
.................
..........
..................
............
